# Setup

In [1]:
import pandas as pd
from pathlib import Path

In [2]:
#input args
log_name = 'logs/LOG_002.csv'
ticks_per_second_mb = 1000
out_dir = './out'

#derived args
split_logs_dir = f'{out_dir}/split_logs/'

In [3]:
#create output folders
Path(f'{out_dir}/split_logs').mkdir(parents=True, exist_ok=True)
Path(f'{out_dir}/plots').mkdir(parents=True, exist_ok=True)

# Full Log

In [4]:
full_log = pd.read_csv(log_name, delimiter = ';')
full_log.index.rename('Log Entry', inplace = True)
full_log.head()

,Timestamp,Log Entry Type,Data
Log Entry,,,
0,500,1,"4,4,219,0,3,65"
1,500,1,"4,4,2415,0,41,718"
2,500,1,"4,4,4610,0,79,1371"
3,600,1,"1,3,0,0,0,0,0,0,0,0,0,0"
4,600,1,"2,3,0,0,0,0,0,0,0,0,0,0"


In [5]:
#convert timestamp ticks to seconds
full_log['Time_s'] = full_log['Timestamp']/ticks_per_second_mb

In [6]:
#change entry type ids to enums
full_log['Log Entry Type'].replace({1:'SENSOR', 2:'STATE', 3:'ESTIMATOR_VAR', 4: 'CONTROLLER_OUTPUT', 5:'MOTOR', 6:'MSG'}, inplace = True)

# Sensor Log

In [7]:
#extract sensor entries
sensor_log = full_log[full_log['Log Entry Type'] == 'SENSOR'].copy()
#extract data for sensor entries
sensor_log[['Sensor Board ID', 'Sensor Type', 'Data']] = sensor_log['Data'].str.split(',', expand = True, n = 2)

#change sensor type ids to enums
sensor_log['Sensor Type'] = sensor_log['Sensor Type'].astype(int)
sensor_log['Sensor Type'].replace({1:'BARO', 2:'IMU', 3:'GPS', 4: 'BATTERY'}, inplace = True)

In [8]:
#extract individual sensor types
baro_log = sensor_log[sensor_log['Sensor Type'] == 'BARO'].copy()
imu_log = sensor_log[sensor_log['Sensor Type'] == 'IMU'].copy()
gps_log = sensor_log[sensor_log['Sensor Type'] == 'GPS'].copy()
battery_log = sensor_log[sensor_log['Sensor Type'] == 'BATTERY'].copy()

## Baro Log

In [9]:
#extract data for baro sensors
baro_log[['Pressure', 'Temperature', 'Sensor Board Timestamp']] = baro_log['Data'].str.split(',', expand = True)
baro_log.drop(columns = ['Data'], inplace = True)
#reorder columns
baro_log = baro_log[['Timestamp', 'Time_s', 'Log Entry Type', 'Sensor Board ID', 'Sensor Board Timestamp', 'Sensor Type', 'Pressure', 'Temperature']]

#convert temperature to Celsius
fmt_tmp = lambda x: x / 100
baro_log['Temperature'] = baro_log['Temperature'].apply(pd.to_numeric).apply(fmt_tmp)
baro_log.head()

,Timestamp,Time_s,Log Entry Type,Sensor Board ID,Sensor Board Timestamp,Sensor Type,Pressure,Temperature
Log Entry,,,,,,,,
10,810,0.81,SENSOR,1,0,BARO,0,0.00
13,810,0.81,SENSOR,2,0,BARO,0,0.00
16,810,0.81,SENSOR,3,0,BARO,0,0.00
20,820,0.82,SENSOR,1,863,BARO,99643,26.44
23,820,0.82,SENSOR,2,863,BARO,99665,25.04


In [10]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
baro_log.to_csv(f'{split_logs_dir}/baro.csv')

## IMU Log

In [11]:
#generate Acc_x, Acc_y, Acc_z, Gyro_x, Gyro_y, Gyro_z
dim3 = lambda x : [x + '_' + s for s in ['x', 'y', 'z']]
imu_cols = dim3('Acc') + dim3('Gyro')

#extract data for imu sensors
imu_log[imu_cols + ['Sensor Board Timestamp']] = imu_log['Data'].str.split(',', expand = True)
imu_log.drop(columns = ['Data'], inplace = True)
#reorder columns
imu_log = imu_log[['Timestamp', 'Time_s', 'Log Entry Type', 'Sensor Board ID', 'Sensor Board Timestamp', 'Sensor Type'] + imu_cols]

#scale acceleration and rotation to float

#convert to gs
fmt_acc = lambda x: x / 1024
#convert to degrees per second
fmt_gyro = lambda x: x / 32.4

imu_log[dim3('Acc')] = imu_log[dim3('Acc')].apply(pd.to_numeric).apply(fmt_acc)
imu_log[dim3('Gyro')] = imu_log[dim3('Gyro')].apply(pd.to_numeric).apply(fmt_gyro)

imu_log.head()

,Timestamp,Time_s,Log Entry Type,Sensor Board ID,Sensor Board Timestamp,Sensor Type,Acc_x,Acc_y,Acc_z,Gyro_x,Gyro_y,Gyro_z
Log Entry,,,,,,,,,,,,
11,810,0.81,SENSOR,1,0,IMU,0.0,0.0,0.0,0.0,0.0,0.0
12,810,0.81,SENSOR,1,0,IMU,0.0,0.0,0.0,0.0,0.0,0.0
14,810,0.81,SENSOR,2,0,IMU,0.0,0.0,0.0,0.0,0.0,0.0
15,810,0.81,SENSOR,2,0,IMU,0.0,0.0,0.0,0.0,0.0,0.0
17,810,0.81,SENSOR,3,0,IMU,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
imu_log.to_csv(f'{split_logs_dir}/imu.csv')

## GPS Log

In [13]:
gps_cols = ['Hour', 'Minute', 'Second', 'Latitude [deg]', 'Latitude [decimal]', 'Longitude [deg]', 'Longitude [decimal]', 'Satellite', 'Altitude', 'HDOP']
gps_log[gps_cols] = gps_log['Data'].str.split(',', expand = True)
gps_log = gps_log[['Timestamp', 'Time_s', 'Log Entry Type', 'Sensor Board ID', 'Sensor Type', ] + gps_cols]
gps_log.head()

,Timestamp,Time_s,Log Entry Type,Sensor Board ID,Sensor Type,Hour,Minute,Second,Latitude [deg],Latitude [decimal],Longitude [deg],Longitude [decimal],Satellite,Altitude,HDOP
Log Entry,,,,,,,,,,,,,,,
3,600,0.6,SENSOR,1,GPS,0,0,0,0,0,0,0,0,0,0
4,600,0.6,SENSOR,2,GPS,0,0,0,0,0,0,0,0,0,0
5,600,0.6,SENSOR,3,GPS,0,0,0,0,0,0,0,0,0,0
6,600,0.6,SENSOR,1,GPS,0,0,0,0,0,0,0,0,0,0
7,600,0.6,SENSOR,2,GPS,0,0,0,0,0,0,0,0,0,0


In [14]:
gps_log.to_csv(f'{split_logs_dir}/gps.csv')

## Battery Log

In [15]:
battery_cols = ['Battery', 'Consumption', 'Current', 'Supply']
battery_log[battery_cols] = battery_log['Data'].str.split(',', expand = True)
battery_log = battery_log[['Timestamp', 'Time_s', 'Log Entry Type', 'Sensor Board ID', 'Sensor Type', ] + battery_cols]
battery_log.head()

,Timestamp,Time_s,Log Entry Type,Sensor Board ID,Sensor Type,Battery,Consumption,Current,Supply
Log Entry,,,,,,,,,
0,500,0.5,SENSOR,4,BATTERY,219,0,3,65
1,500,0.5,SENSOR,4,BATTERY,2415,0,41,718
2,500,0.5,SENSOR,4,BATTERY,4610,0,79,1371
9,600,0.6,SENSOR,4,BATTERY,6806,0,115,2024
19,800,0.8,SENSOR,4,BATTERY,9003,0,149,2675


In [16]:
battery_log.to_csv(f'{split_logs_dir}/battery.csv')

# State Log

In [17]:
#IDLE = 1, AIRBRAKE_TEST, THRUSTING, COASTING, DESCENT, RECOVERY

In [18]:
#extract state change entries
state_log = full_log[full_log['Log Entry Type'] == 'STATE'].copy()
state_log.rename(columns={'Data': 'State'}, inplace = True)
state_log['State'] = state_log['State'].astype(int)
state_log['State'].replace({1:'IDLE', 2:'AIRBRAKE_TEST', 3:'THRUSTING', 4:'COASTING', 5:'DESCENT', 6:'RECOVERY'}, inplace = True)
state_log.head()

,Timestamp,Log Entry Type,State,Time_s
Log Entry,,,,
7830,10000,STATE,IDLE,10.00
7842,10010,STATE,IDLE,10.01
7853,10020,STATE,IDLE,10.02
7864,10030,STATE,IDLE,10.03
7875,10040,STATE,IDLE,10.04


In [19]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
state_log.to_csv(f'{split_logs_dir}/state_transitions.csv')

# Estimator Variable Log

In [20]:
#extract estimator var entries
estimator_var_log = full_log[full_log['Log Entry Type'] == 'ESTIMATOR_VAR'].copy()
estimator_var_log[['World Position (z)', 'Rocket Velocity (z)',  'Rocket Acceleration (z)']] = estimator_var_log['Data'].str.split(',', expand = True)

estimator_var_log.drop(columns = ['Data'], inplace = True)
#reorder columns
estimator_var_log = estimator_var_log[['Timestamp', 'Time_s', 'Log Entry Type', 'World Position (z)', 'Rocket Velocity (z)', 'Rocket Acceleration (z)']]

fmt_est = lambda x: x / 1000
estimator_var_log[['World Position (z)', 'Rocket Velocity (z)',  'Rocket Acceleration (z)']] = estimator_var_log[['World Position (z)', 'Rocket Velocity (z)',  'Rocket Acceleration (z)']].apply(pd.to_numeric).apply(fmt_est)
estimator_var_log.head()

,Timestamp,Time_s,Log Entry Type,World Position (z),Rocket Velocity (z),Rocket Acceleration (z)
Log Entry,,,,,,
110,900,0.90,ESTIMATOR_VAR,0.0,0.000,0.0
120,910,0.91,ESTIMATOR_VAR,0.0,0.000,0.0
130,920,0.92,ESTIMATOR_VAR,0.0,0.000,0.0
140,930,0.93,ESTIMATOR_VAR,0.0,0.001,0.0
150,940,0.94,ESTIMATOR_VAR,0.0,0.003,0.0


In [21]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
estimator_var_log.to_csv(f'{split_logs_dir}/estimator_var.csv')

# Controller Output Log

In [22]:
#extract estimator var entries
controller_out_log = full_log[full_log['Log Entry Type'] == 'CONTROLLER_OUTPUT'].copy()
controller_out_log[['Controller Output', 'Reference Error', 'Integrated Error']] = controller_out_log['Data'].str.split(',', expand = True)

controller_out_log.drop(columns = ['Data'], inplace = True)
#reorder columns
controller_out_log = controller_out_log[['Timestamp', 'Time_s', 'Log Entry Type', 'Controller Output', 'Reference Error', 'Integrated Error']]

fmt_cnt = lambda x: x / 1000
controller_out_log[['Controller Output', 'Reference Error', 'Integrated Error']] = controller_out_log[['Controller Output', 'Reference Error', 'Integrated Error']].apply(pd.to_numeric).apply(fmt_cnt)
controller_out_log.head()

ValueError: Columns must be same length as key

In [105]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
controller_out_log.to_csv(f'{split_logs_dir}/controller_out.csv')

# Message Log

In [23]:
#extract message entries
msg_log = full_log[full_log['Log Entry Type'] == 'MOTOR'].copy()
msg_log.head()

,Timestamp,Log Entry Type,Data,Time_s
Log Entry,,,,
1748,3080,MOTOR,"0,0",3.08
1789,3120,MOTOR,"0,0",3.12
1830,3160,MOTOR,"0,0",3.16
1871,3200,MOTOR,"0,0",3.20
1912,3240,MOTOR,"0,0",3.24


In [24]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
msg_log.to_csv(f'{split_logs_dir}/motor.csv')

In [25]:
#extract message entries
msg_log = full_log[full_log['Log Entry Type'] == 'MSG'].copy()
msg_log.head()

,Timestamp,Log Entry Type,Data,Time_s
Log Entry,,,,


In [26]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
msg_log.to_csv(f'{split_logs_dir}/messages.csv')